# Plug flow reactor simulation of Thruster

![caption](Graphics/thruster-details.png)


In [25]:
import cantera as ct
import numpy as np

from matplotlib import pyplot as plt
import csv
import os
import re
import itertools
import pandas as pd
from collections import defaultdict

In [26]:
# default if not using SLURM array
cat_area_per_vol = 3e6 # m2/m3
temperature_c = 550 # ºC
rtol = 1e-11
atol = 1e-24
residual_threshold = 5e-3

# input file containing the reaction mechanism
cti_file = '../RMG-model/cantera/chem_annotated.cti'
#cti_file = '../RMG-model/cantera/chem0050.cti'

In [27]:
cat_area_per_vol_options = [3e2, 3e3, 3e4, 3e5, 6e5, 9e5, 1.2e6, 3e6, 3e7, 3e8] # m2/m3
temperature_c_options = [200, 300, 400, 500, 600] # ºC

cat_area_per_vol_options = [3e3, 3e4, 3e5, 3e6, 3e7, 3e8] # m2/m3
temperature_c_options = [200, 300, 400, 500] # ºC
rtol_options = [1e-6, 1e-9, 1e-11]
atol_options = [1e-15, 1e-18, 1e-21, 1e-24]

settings  = list(itertools.product(cat_area_per_vol_options,
                                   temperature_c_options,
                                   rtol_options,
                                   atol_options
                                  ))
print(f"Settings array is from 0 to {len(settings)-1} ")

Settings array is from 0 to 287 


In [28]:
with open('rocketman_array.sh') as f:
    for l in f:
        m = re.match('#SBATCH --array=(\d+)-(\d+)', l)
        if m:
            print(l)
            start = int(m.group(1))
            end = int(m.group(2))
assert (start, end) == (0, len(settings)-1 ), "SLURM array doesn't match settings array"

#SBATCH --array=0-287%100



In [29]:
print(f"Catalyst area per volume {cat_area_per_vol :.2e} m2/m3")
print(f"Initial temperature      {temperature_c :.1f} ºC")
print(f"Solver RTOL              {rtol :.1e}")
print(f"Solvel ATOL              {atol :.1e}")

Catalyst area per volume 3.00e+06 m2/m3
Initial temperature      550.0 ºC
Solver RTOL              1.0e-11
Solvel ATOL              1.0e-24


In [30]:
gas=ct.Solution(cti_file)
surf = ct.Interface(cti_file,'surface1', [gas])

In [31]:
gas()


  gas:

       temperature             300  K
          pressure          101325  Pa
           density         0.81974  kg/m^3
  mean mol. weight         20.1797  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy          1905.6        3.845e+04     J
   internal energy      -1.217e+05       -2.456e+06     J
           entropy          7257.7        1.465e+05     J/K
    Gibbs function     -2.1754e+06        -4.39e+07     J
 heat capacity c_p          1030.1        2.079e+04     J/K
 heat capacity c_v          618.03        1.247e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                Ne              1                1         -17.5994
     [ +186 minor]              0                0



In [32]:
print(", ".join(gas.species_names))

Ne, NH3(2), NH2OH(3), HNO3(4), CH3OH(5), H2O(6), N2(7), O2(8), NO2(9), NO(10), N2O(11), CO2(12), H2(13), CO(14), CH4(15), C2H6(16), CH2O(17), CH3(18), C3H8(19), H(20), C2H5(21), HCO(22), CH3CHO(23), OH(24), C2H4(25), O(36), Ar(37), HO2(39), H2O2(40), HOCO(41), CH2(42), CH2(S)(43), CH(44), CH2OH(45), CH3O(46), HCOH(47), CH3OO(48), CH2CO(49), C2H3(50), C(51), C2H2(52), C2H(53), CH3OOH(54), CH2OOH(55), HOCH2O(56), HOCHO(57), C2H5O(58), C2H5O2(59), C2H5O2(60), cC2H4O(61), CH2CHO(62), H2CC(63), CH3CO(64), C2H4O(65), C2H5O(66), C2H3O2(67), CHCHO(68), OCHCHO(69), HCCO(70), HCCOH(71), CHCHOH(72), C2(73), C2O(74), C2H6O(75), C2H5O(76), C2H5O3(77), cC2H3O(78), C2H3O3(79), OCHCO(80), C2H6O2(81), C2H5O2(82), C2H4O2(83), OCHO(84), NH2(85), NH(86), HNO(87), H2NO(88), HON(89), N(90), NNH(91), HONO(92), HNOH(93), HNO2(94), NO3(95), N2H2(96), H2N2(97), N2H3(98), N2H4(99), HCN(100), CN(101), HNC(102), NCO(103), HOCN(104), HNCO(105), NCCN(106), HNCN(107), NCN(108), HNCNH(109), HCNO(110), CH3CN(111), CH2C

In [33]:
print(", ".join(surf.species_names))

X(1), HX(26), OX(27), CH3X(28), HOX(29), H2OX(30), CO2X(31), OCX(32), CX(33), CH2X(34), CHX(35), H2NX(201), H3NX(202), H2NOX(203), H2NOX(204), H3NOX(205), NO2X(206), NO3X(207), HNO3X(208), CH3OX(209), CH3OX(210), CH4OX(211), OX(213), NOX(215), NO2X(216), NOJX(217), H2X(219), SX(494), SX(495), CHO2X(506), HNO2X(513), HNX(532), HNOX(533), NX(604), SX(605), SX(606), SX(609), H2N2X(614), N2O3X(618), SX(623), HNOX(631), H2NOX(633), HNO2X(741), SX(759), HONOX(934), SX(1000), SX(1039), SX(1043), SX(1047), SX(1048), SX(1050), SX(1063), SX(1064), HNOX(1071), SX(1163), SX(1249), N2OX(1419), CNOX(1438), SX(1527), SX(1895), SX(2232), SX(2233), SX(2465), SX(3049), SX(3050), SX(3248), SX(3939), SX(4539), SX(4540), NO3X(4718), SX(4793), SX(4794), SX(4937), SX(5513), SX(5810), SX(5814), SX(6977), SX(7604), SX(9643)




This example solves a plug flow reactor problem, with coupled surface and gas chemistry.





In [34]:
# unit conversion factors to SI
cm = 0.01 # m
minute = 60.0  # s

In [35]:
#######################################################################
# Input Parameters for combustor
#######################################################################
mass_flow_rate =  0.5e-3 # kg/s
#temperature_c = 550.0  # Initial Temperature in Celsius
print(f"Initial temperature {temperature_c :.1f} ºC")
pressure = ct.one_atm # constant

length = 1.1 * cm  # Catalyst bed length. 11mm
cross_section_area = np.pi * (0.9*cm)**2  # Catalyst bed area.  18mm diameter circle.

### Catalyst properties. Some are hard to estimate
# if we can, update this lit value or verify the value richard calculated
porosity = 0.38  # Catalyst bed porosity (0.38)
# Al2O3 particles are about 0.7mm diameter
cat_specific_area = 140 # m2/g
print(f"Catalyst specific area {cat_specific_area :.2e} m2/g")
cat_density = 2 / cm**3 # 2 g/m3
print(f"Catalyst density {cat_density :.2e} g/m3")
cat_area_per_reactor_vol = cat_specific_area * cat_density # m2/m3
print(f"Catalyst area per total reactor volume {cat_area_per_reactor_vol :.2e} m2/m3")
cat_area_per_gas_vol = cat_area_per_reactor_vol / porosity # porosity is gas vol per reactor vol
print(f"Catalyst area per gas volume {cat_area_per_gas_vol :.2e} m/m3")

cat_area_per_vol2 =  cat_area_per_gas_vol * 1e-3 # REDUCE BY A LOT
print(f"{cat_area_per_vol2:.2e}")
cat_area_per_vol = 2.80e+05
# cat_area_per_vol = 5314.2857

print(f"\nCatalyst area per volume in use for this simulation: {cat_area_per_vol :.2e} m2/m3")

Initial temperature 550.0 ºC
Catalyst specific area 1.40e+02 m2/g
Catalyst density 2.00e+06 g/m3
Catalyst area per total reactor volume 2.80e+08 m2/m3
Catalyst area per gas volume 7.37e+08 m/m3
7.37e+05

Catalyst area per volume in use for this simulation: 5.31e+03 m2/m3


In [36]:
output_filename = 'surf_pfr_output_mlou_test.csv'

# The PFR will be simulated by a chain of 'NReactors' stirred reactors.
NReactors = 2201

#####################################################################

temperature_kelvin = temperature_c + 273.15  # convert to Kelvin

# import the gas model and set the initial conditions
gas = ct.Solution(cti_file, 'gas')

# From HAN-molefractions.ipynb
feed_mole_fractions = {
    'NH3(2)': 0.031,
    'NH2OH(3)': 0.32,
    'HNO3(4)': 0.35,
    'CH3OH(5)': 0.21,
    'H2O(6)': 0.09,
}
gas.TPX = temperature_kelvin, pressure, feed_mole_fractions


# import the surface model
surf = ct.Interface(cti_file,'surface1', [gas])
surf.TP = temperature_kelvin, pressure
surf.coverages = 'X(1):1.0'

r_len = length/(NReactors-1) 
r_vol = cross_section_area * r_len * porosity # gas volume

# catalyst area in one reactor
cat_area = cat_area_per_vol * r_vol

# Not sure we need the velocity
velocity = mass_flow_rate / (gas.density * cross_section_area)

In [37]:
def report_rates(n=8):
    print("\nHighest net rates of progress, gas")
    for i in np.argsort(abs(gas.net_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.net_rates_of_progress[i]:8.1g}")
    print("\nHighest net rates of progress, surface")
    for i in np.argsort(abs(surf.net_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.net_rates_of_progress[i]:8.1g}")
    print("\nHighest forward rates of progress, gas")
    for i in np.argsort(abs(gas.forward_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.forward_rates_of_progress[i]:8.1g}")
    print("\nHighest forward rates of progress, surface")
    for i in np.argsort(abs(surf.forward_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.forward_rates_of_progress[i]:8.1g}")
    print("\nHighest reverse rates of progress, gas")
    for i in np.argsort(abs(gas.reverse_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.reverse_rates_of_progress[i]:8.1g}")
    print("\nHighest reverse rates of progress, surface")
    for i in np.argsort(abs(surf.reverse_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.reverse_rates_of_progress[i]:8.1g}")

    print(f"\nSurface rates have been scaled by surface/volume ratio {cat_area_per_vol:.1e} m2/m3")
    print("So are on a similar basis of volume of gas")
    print(" kmol / m3 / s")
report_rates()


Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.8
275 : NH2OH(3) (+M) <=> NH2(85) + OH(24) (+M)              2e-05
 88 : H2NO(88) + H2NO3(243) <=> HNO3(4) + NH2OH(3)        -4e-06
 61 : H(20) + H2NO(88) <=> NH2OH(3)                       -1e-07
 87 : H2NO3(243) + HNOH(93) <=> HNO3(4) + NH2OH(3)        -2e-09
 70 : H(20) + HNOH(93) <=> NH2OH(3)                       -2e-10
 42 : CH3(18) + OH(24) <=> CH3OH(5)                       -7e-12

Highest net rates of progress, surface
  0 : H2O(6) + X(1) <=> H2OX(30)                           1e+03
  3 : NH2OH(3) + X(1) <=> H3NOX(205)                       5e+02
  4 : HNO3(4) + X(1) <=> HNO3X(208)                        4e+02
  5 : CH3OH(5) + X(1) <=> CH4OX(211)                       3e+02
  6 : NH2OH(3) + 2 X(1) <=> H2NOX(204) + HX(26)            1e+02
 71 : CH3OH(5) + 2 X(1) <=> CH3OX(210) + HX(26)            1e+02
  8 : NH2OH(3) + 2 X(1) <=> H2NOX(203) + HX(26)            7e+01

Highest forwa

In [38]:
def report_rate_constants(n=8):
    print("\nHighest forward rate constants, gas")
    for i in np.argsort(abs(gas.forward_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.forward_rate_constants[i]:8.1e}")
    print("\nHighest forward rate constants, surface")
    for i in np.argsort(abs(surf.forward_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {surf.forward_rate_constants[i]:8.1e}")
    print("\nHighest reverse rate constants, gas")
    for i in np.argsort(abs(gas.reverse_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.reverse_rate_constants[i]:8.1e}")
    print("\nHighest reverse rate constants, surface")
    for i in np.argsort(abs(surf.reverse_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {surf.reverse_rate_constants[i]:8.1e}")

    print("Units are a combination of kmol, m^3 and s, that depend on the rate expression for the reaction.")
report_rate_constants()


Highest forward rate constants, gas
295 : CH2OH(45) + H2NO(88) <=> S(1108)                   3.7e+15
382 : HCO(22) + NO(10) <=> CHNO2(397)                    3.7e+15
474 : HCO(22) + HNOH(93) <=> ONCDO(266)                  3.7e+15
479 : HCO(22) + HNO(630) <=> S(665)                      3.7e+15
296 : CH2OH(45) + NO(10) <=> S(1494)                     3.7e+15
241 : CH2OH(45) + NO2(9) <=> S(1216)                     3.7e+15
596 : HCO(22) + NO2(9) <=> CHNO3(372)                    3.7e+15

Highest forward rate constants, surface
467 : CH2X(34) + HX(26) <=> CH3X(28) + X(1)              3.1e+21
 83 : CHX(35) + HX(26) <=> CH2X(34) + X(1)               3.1e+21
 94 : CHX(35) + X(1) <=> CX(33) + HX(26)                 3.1e+21
141 : SX(1039) + X(1) <=> HX(26) + SX(1047)              3.7e+20
745 : SX(1043) + X(1) <=> HX(26) + SX(1050)              3.7e+20
358 : SX(1064) + X(1) <=> H2NX(201) + SX(1000)           6.8e+19
683 : SX(4937) + X(1) <=> H2NOX(204) + OCX(32)           6.8e+19

Highest rev

In [39]:
def fix_rates(phase, limit):
    """
    Fix reverse reaction rates that are too fast.
    """
    for i in np.argsort(abs(phase.reverse_rate_constants))[-1:0:-1]:
        if phase.reverse_rate_constants[i] < limit:
            break
        print(f"Before: {i:3d} : {phase.reaction_equation(i):48s}  {phase.reverse_rate_constants[i]:8.1e}")
        multiplier = limit / phase.reverse_rate_constants[i]
        phase.set_multiplier(multiplier, i)
        print(f"After:  {i:3d} : {phase.reaction_equation(i):48s}  {phase.reverse_rate_constants[i]:8.1e}")
        
#fix_rates(gas, 1e18)
#fix_rates(surf, 1e21)

In [41]:
print("\nHighest forward rate constants, surface")
for i in np.argsort(abs(surf.forward_rate_constants))[-1:-20:-1]: # top n in descending order
    print(f"{i:3d} : {surf.reaction_equation(i):48s}  {surf.forward_rate_constants[i]:8.1e}")



Highest forward rate constants, surface
467 : CH2X(34) + HX(26) <=> CH3X(28) + X(1)              3.1e+21
 83 : CHX(35) + HX(26) <=> CH2X(34) + X(1)               3.1e+21
 94 : CHX(35) + X(1) <=> CX(33) + HX(26)                 3.1e+21
141 : SX(1039) + X(1) <=> HX(26) + SX(1047)              3.7e+20
745 : SX(1043) + X(1) <=> HX(26) + SX(1050)              3.7e+20
358 : SX(1064) + X(1) <=> H2NX(201) + SX(1000)           6.8e+19
683 : SX(4937) + X(1) <=> H2NOX(204) + OCX(32)           6.8e+19
134 : SX(1039) + X(1) <=> CHX(35) + H2NX(201)            6.8e+19
164 : CHX(35) + SX(1063) <=> 2 SX(1000)                  3.5e+19
146 : CH2X(34) + SX(1047) <=> CHX(35) + SX(1039)         3.5e+19
340 : CHX(35) + SX(605) <=> HNOX(533) + SX(1000)         3.5e+19
119 : CH2X(34) + SX(1000) <=> CH3OX(210) + CHX(35)       3.4e+19
339 : CH2X(34) + SX(605) <=> CH3OX(210) + HNOX(533)      3.4e+19
373 : SX(1039) + SX(605) <=> HNOX(533) + SX(1064)        3.4e+19
113 : H2NX(201) + SX(1000) <=> CH3OX(210) + HNX(5

In [42]:
def save_flux_diagrams(*phases, suffix=''):
    """
    Saves the flux diagrams. The filenames have a suffix if provided,
    so you can keep them separate and not over-write.
    """
    for element in 'CHONX':
        for phase_object in phases:
            phase = phase_object.name

            diagram = ct.ReactionPathDiagram(phase_object, element)
            diagram.title = f'Reaction path diagram following {element} in {phase}'
            diagram.label_threshold = 0.01

            dot_file = f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.dot"
            img_file = f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.png"
            img_path = os.path.join(os.getcwd(), img_file)
            diagram.write_dot(dot_file)
            #print(diagram.get_data())

            print(f"Wrote graphviz input file to '{os.path.join(os.getcwd(), dot_file)}'.")
            os.system(f'dot {dot_file} -Tpng -o{img_file} -Gdpi=200')
            print(f"Wrote graphviz output file to '{img_path}'.")

def show_flux_diagrams(*phases, suffix='', embed=False):
    """
    Shows the flux diagrams in the notebook.
    Loads them from disk.
    Does not embed them, to keep the .ipynb file small,
    unless embed=True. Use embed=True if you might over-write the files,
    eg. you want to show flux at different points.
    """
    import IPython
    for element in 'CHONX':
        for phase_object in phases:
            phase = phase_object.name
            img_file = f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.png"
            display(IPython.display.HTML(f'<hr><h2>{element} {phase}</h2>'))
            if embed:
                display(IPython.display.Image(filename=img_file,width=400,embed=True))
            else:
                display(IPython.display.Image(url=img_file,width=400,embed=False))


def integrated_flux_diagrams():
    """This is a code fragment. Not working. Do not use it."""
    for element in 'CHON':
        diagrams = [ct.ReactionPathDiagram(surf, element), ct.ReactionPathDiagram(gas, element)]
        for diagram in diagrams:
            data = diagram.get_data()
            split_data = data.split("\n")
            for line in split_data[2:]:
                if len(line.split()) == 0: # skip empty line
                    continue
                s1, s2, fwd, rev = line.split()
                net = float(fwd) - float(rev)
                if net == 0.0:
                    continue
                flux_pair = (s1, s2)
                integration_flux_data[flux_pair] += net
    

In [43]:
gas.TPX = temperature_kelvin, pressure, feed_mole_fractions
surf.coverages = 'X(1):1.0'
#surf.coverages = starting_coverages

In [44]:
def remove_heat(z):
    '''
    for now, just makes tempertuare 5% less than it should be at each reactor 
    perahps later can adjust this based on temp. profiles from literature / 
    a priori eqns
    '''
    factor = 0.05
    new_T = (1-factor)*gas.T
    return new_T

In [45]:
wall_A = r_len*np.pi*1.8*cm


In [ ]:
# The plug flow reactor is represented by a linear chain of zero-dimensional
# reactors. The gas at the inlet to the first one has the specified inlet
# composition, and for all others the inlet composition is fixed at the
# composition of the reactor immediately upstream. Since in a PFR model there
# is no diffusion, the upstream reactors are not affected by any downstream
# reactors, and therefore the problem may be solved by simply marching from
# the first to last reactor, integrating each one to steady state.

TDY = gas.TDY
cov = surf.coverages

# create a new reactor
gas.TDY = TDY
r = ct.IdealGasReactor(gas, energy='on')
r.volume = r_vol

# create a reservoir to represent the reactor immediately upstream. Note
# that the gas object is set already to the state of the upstream reactor
upstream = ct.Reservoir(gas, name='upstream')

# create a reservoir for the reactor to exhaust into. The composition of
# this reservoir is irrelevant.
downstream = ct.Reservoir(gas, name='downstream')

# Add the reacting surface to the reactor. The area is set to the desired
# catalyst area in the reactor.
rsurf = ct.ReactorSurface(surf, r, A=cat_area)

# The mass flow rate into the reactor will be fixed by using a
# MassFlowController object.
m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)

# We need an outlet to the downstream reservoir. This will determine the
# pressure in the reactor. The value of K will only affect the transient
# pressure difference.
v = ct.PressureController(r, downstream, master=m, K=1e-5)

# create a reservoir for the environment, and fill it with air.
env = ct.Reservoir(ct.Solution('air.xml'))

# heat loss to the environment. Heat loss always occur through walls, so we
# create a wall separating r1 from the environment, give it a non-zero area,
# and specify the overall heat transfer coefficient through the wall.
wall_A = r_len*np.pi*1.8*cm
w2 = ct.Wall(r, env, A=wall_A, U=500.0)

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 24

# set relative and absolute tolerances on the simulation
sim.rtol = rtol
sim.atol = atol

sim.verbose = False

# surf.set_multiplier(0.)  # turn off surface reactions
# surf.set_multiplier(1e6)  # make surface reactions a million times faster

r.volume = r_vol
# rsurf.area = cat_area

integration_flux_data = defaultdict(float)

outfile = open(output_filename,'w')
writer = csv.writer(outfile)
writer.writerow(['Distance (mm)', 'T (C)', 'P (atm)'] +
                gas.species_names + surf.species_names + ['gas_heat','surface_heat','alpha'])

print('    distance(mm)     T (C)    NH3(2)   NH2OH(3)     HNO3(4)    CH3OH(5)  alpha')
for n in range(NReactors):
    """
    if n == 0: # first coulpe of reactors are tiny
        surf.set_multiplier(0.)
        r.volume = r_vol * 1e-2
        rsurf.area = cat_area * 1e-2
    if n == 3:
        r.volume = r_vol
        rsurf.area = cat_area"""
        
#     if n == 0: # start off with inert packing, no surface reactions
#         surf.set_multiplier(0.)
#     if n == int(0.001 * NReactors / length): # after 1 mm, catalyst
#         surf.set_multiplier(1)
        
#         fix_rates(surf, 1e22)
#         save_flux_diagrams(gas, surf, suffix='1mm')
#         show_flux_diagrams(gas, surf, suffix='1mm', embed=True)
    
    # Set the state of the reservoir to match that of the previous reactor
    gas.TDY = TDY = r.thermo.TDY 
#     print(f"after TDY = f.thermo.TDY: T= {gas.T:.0f}")
#     cov = surf.coverages
    upstream.syncState()
    sim.reinitialize()
#     print(f"after reinitialize: T= {gas.T:.0f}")
    try:
#       the default is residual_threshold = sim.rtol*10
#         sim.advance_to_steady_state(residual_threshold = residual_threshold)
        sim.advance_to_steady_state()
#         print(f"after advance_to_steady_state: T= {gas.T:.0f}")
    except ct.CanteraError:
        t = sim.time
        sim.set_initial_time(0)
        gas.TDY = TDY
        surf.coverages = cov
        r.syncState()
        sim.reinitialize()
        new_target_time = 0.01 * t
        print(f"Couldn't reach {t:.1g} s so going to try {new_target_time:.1g} s")
        #save_flux_diagrams(gas, surf)
        #show_flux_diagrams(gas, surf, embed=True)
        report_rates()
        report_rate_constants()
        try:
            sim.advance(new_target_time)
        except ct.CanteraError:
            outfile.close()
            raise()
            #report_rate_constants()
 
    dist = n * r_len * 1.0e3   # distance in mm
        
    gas_heat = np.dot(gas.net_rates_of_progress, gas.delta_enthalpy) # heat evolved by gas phase reaction
    surface_heat = cat_area_per_vol * np.dot(surf.net_rates_of_progress, surf.delta_enthalpy) # heat evolved by surf phase reaction 
    alpha = surface_heat / (surface_heat + gas_heat) # fraction of heat release that is on surface.

#     #########################################
#     # heat transfer
#     #########################################
#     print(f"before set: T= {gas.T:.0f}")
#     T_colder = remove_heat(n)
# #     gas.TP = T_colder, pressure #This didn't work- need to set reactor temp?
#     r.T = T_colder
#     print(f"after set: T= {gas.T:.0f}")
    
    if not n % 10:
        print('    {:10f}  {:7.1f}  {:10f}  {:10f}  {:10f} {:10f}  {:5.1e}'.format(dist, r.T-273.15, *gas['NH3(2)','NH2OH(3)','HNO3(4)','CH3OH(5)'].X, alpha ))

    # write the gas mole fractions and surface coverages vs. distance
    writer.writerow([dist, r.T - 273.15, r.thermo.P/ct.one_atm] +
                    list(gas.X) + list(surf.coverages) + [gas_heat, surface_heat, alpha])
    

outfile.close()
print("Results saved to '{0}'".format(output_filename))

# with open("integration_flux_data.txt",'w') as f:
#     for (sp1,sp2),flux in integration_flux_data.items():
#         f.write("{} {} {}\n".format(sp1,sp2,flux))
            

    distance(mm)     T (C)    NH3(2)   NH2OH(3)     HNO3(4)    CH3OH(5)  alpha
      0.000000    549.9    0.030968    0.319631    0.349610   0.209779  -1.6e-13
Couldn't reach 5e+09 s so going to try 5e+07 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.8
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.5
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.4
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1
 60 : NH2OH(3) + OH(24) <=> H2NO(88) + H2O(6)                0.1
437 : CH2OH(45) + HNOH(93) <=> ONCO(1021)                   0.07
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)           0.07

Highest net rates of progress, surface
410 : NX(604) + SX(606) <=> H2N2X(614) + HNOX(533)           nan
384 : CX(33) + HNO3X(208) <=> CHX(35) + NO3X(207)            nan
382 : HNO3X(208) + HNX(532) <=> H2NX(201) + NO3X(207)        nan
380 : HNO3X(208) + HNOX(533) <=> H2NOX(204) + NO3X(207)       nan
3

Couldn't reach 2e+09 s so going to try 2e+07 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.8
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.7
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.5
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)            0.2
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1
 60 : NH2OH(3) + OH(24) <=> H2NO(88) + H2O(6)                0.1
145 : HNO2(94) (+M) <=> HONO(92) (+M)                        0.1

Highest net rates of progress, surface
410 : NX(604) + SX(606) <=> H2N2X(614) + HNOX(533)           nan
384 : CX(33) + HNO3X(208) <=> CHX(35) + NO3X(207)            nan
382 : HNO3X(208) + HNX(532) <=> H2NX(201) + NO3X(207)        nan
380 : HNO3X(208) + HNOX(533) <=> H2NOX(204) + NO3X(207)       nan
379 : HNO3X(208) + OX(27) <=> HOX(29) + NO3X(207)            nan
378 : HNO3X(208) + X(1) <=> HX(26) + NO3X(207)               nan
375 : CHX(35) + SX(1064) <=> SX

Couldn't reach 2e+09 s so going to try 2e+07 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.8
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.7
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.5
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)            0.2
145 : HNO2(94) (+M) <=> HONO(92) (+M)                        0.2
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1
 60 : NH2OH(3) + OH(24) <=> H2NO(88) + H2O(6)                0.1

Highest net rates of progress, surface
410 : NX(604) + SX(606) <=> H2N2X(614) + HNOX(533)           nan
320 : HONOX(934) + SX(3050) <=> NO2X(216) + SX(2232)         nan
391 : HNO3X(208) + SX(1527) <=> NO3X(207) + SX(1063)         nan
394 : HNO3X(208) + HNOX(1071) <=> H2NOX(633) + NO3X(207)       nan
395 : HNOX(1071) + NO3X(207) <=> OX(27) + SX(2232)           nan
396 : HNO3X(208) + SX(3050) <=> NO3X(207) + SX(2232)         nan
398 : NO3X(207) + SX(3050) <=>

Couldn't reach 3e+08 s so going to try 3e+06 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.8
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.7
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.5
145 : HNO2(94) (+M) <=> HONO(92) (+M)                        0.3
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)            0.3
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1
 60 : NH2OH(3) + OH(24) <=> H2NO(88) + H2O(6)                0.1

Highest net rates of progress, surface
410 : NX(604) + SX(606) <=> H2N2X(614) + HNOX(533)           nan
384 : CX(33) + HNO3X(208) <=> CHX(35) + NO3X(207)            nan
382 : HNO3X(208) + HNX(532) <=> H2NX(201) + NO3X(207)        nan
380 : HNO3X(208) + HNOX(533) <=> H2NOX(204) + NO3X(207)       nan
379 : HNO3X(208) + OX(27) <=> HOX(29) + NO3X(207)            nan
378 : HNO3X(208) + X(1) <=> HX(26) + NO3X(207)               nan
375 : CHX(35) + SX(1064) <=> SX

Couldn't reach 7e+07 s so going to try 7e+05 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.8
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.7
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.5
145 : HNO2(94) (+M) <=> HONO(92) (+M)                        0.3
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)            0.3
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1
 60 : NH2OH(3) + OH(24) <=> H2NO(88) + H2O(6)                0.1

Highest net rates of progress, surface
410 : NX(604) + SX(606) <=> H2N2X(614) + HNOX(533)           nan
384 : CX(33) + HNO3X(208) <=> CHX(35) + NO3X(207)            nan
382 : HNO3X(208) + HNX(532) <=> H2NX(201) + NO3X(207)        nan
380 : HNO3X(208) + HNOX(533) <=> H2NOX(204) + NO3X(207)       nan
379 : HNO3X(208) + OX(27) <=> HOX(29) + NO3X(207)            nan
378 : HNO3X(208) + X(1) <=> HX(26) + NO3X(207)               nan
375 : CHX(35) + SX(1064) <=> SX

...

      0.100000    548.2    0.030953    0.318458    0.348836   0.209567  1.1e-13
Couldn't reach 3e+06 s so going to try 3e+04 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.7
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.7
145 : HNO2(94) (+M) <=> HONO(92) (+M)                        0.5
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.4
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)            0.2
120 : 2 H2NO(88) <=> HNO(87) + NH2OH(3)                     -0.1
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1

Highest net rates of progress, surface
  0 : H2O(6) + X(1) <=> H2OX(30)                           1e+03
  3 : NH2OH(3) + X(1) <=> H3NOX(205)                       5e+02
  4 : HNO3(4) + X(1) <=> HNO3X(208)                        4e+02
  5 : CH3OH(5) + X(1) <=> CH4OX(211)                       3e+02
 71 : CH3OH(5) + 2 X(1) <=> CH3OX(210) + HX(26)            1e+02
  6 : NH2OH(3) + 

Couldn't reach 4e+06 s so going to try 4e+04 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.7
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.7
145 : HNO2(94) (+M) <=> HONO(92) (+M)                        0.6
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.4
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)            0.2
120 : 2 H2NO(88) <=> HNO(87) + NH2OH(3)                     -0.1
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1

Highest net rates of progress, surface
  0 : H2O(6) + X(1) <=> H2OX(30)                           1e+03
  3 : NH2OH(3) + X(1) <=> H3NOX(205)                       5e+02
  4 : HNO3(4) + X(1) <=> HNO3X(208)                        4e+02
  5 : CH3OH(5) + X(1) <=> CH4OX(211)                       3e+02
 71 : CH3OH(5) + 2 X(1) <=> CH3OX(210) + HX(26)            1e+02
  6 : NH2OH(3) + 2 X(1) <=> H2NOX(204) + HX(26)            1e+02
  8 : NH2OH(3) + 2 X(1) <=> H2NO

Couldn't reach 4e+06 s so going to try 4e+04 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.7
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.7
145 : HNO2(94) (+M) <=> HONO(92) (+M)                        0.6
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.4
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)            0.2
120 : 2 H2NO(88) <=> HNO(87) + NH2OH(3)                     -0.1
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1

Highest net rates of progress, surface
  0 : H2O(6) + X(1) <=> H2OX(30)                           1e+03
  3 : NH2OH(3) + X(1) <=> H3NOX(205)                       5e+02
  4 : HNO3(4) + X(1) <=> HNO3X(208)                        4e+02
  5 : CH3OH(5) + X(1) <=> CH4OX(211)                       3e+02
 71 : CH3OH(5) + 2 X(1) <=> CH3OX(210) + HX(26)            1e+02
  6 : NH2OH(3) + 2 X(1) <=> H2NOX(204) + HX(26)            1e+02
 42 : NH2OH(3) + 2 X(1) <=> H2NX

Couldn't reach 4e+06 s so going to try 4e+04 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.7
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.7
145 : HNO2(94) (+M) <=> HONO(92) (+M)                        0.6
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.4
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)            0.2
120 : 2 H2NO(88) <=> HNO(87) + NH2OH(3)                     -0.1
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1

Highest net rates of progress, surface
  0 : H2O(6) + X(1) <=> H2OX(30)                           1e+03
  3 : NH2OH(3) + X(1) <=> H3NOX(205)                       5e+02
  4 : HNO3(4) + X(1) <=> HNO3X(208)                        4e+02
  5 : CH3OH(5) + X(1) <=> CH4OX(211)                       3e+02
 71 : CH3OH(5) + 2 X(1) <=> CH3OX(210) + HX(26)            1e+02
  6 : NH2OH(3) + 2 X(1) <=> H2NOX(204) + HX(26)            1e+02
  8 : NH2OH(3) + 2 X(1) <=> H2NO

Couldn't reach 4e+06 s so going to try 4e+04 s

Highest net rates of progress, gas
 47 : NO2(9) + OH(24) (+M) <=> HNO3(4) (+M)                 -0.7
 82 : H2NO(88) + HNO2(94) <=> NH2OH(3) + NO2(9)             -0.7
145 : HNO2(94) (+M) <=> HONO(92) (+M)                        0.6
 68 : NH2OH(3) + OH(24) <=> H2O(6) + HNOH(93)                0.4
 76 : HNOH(93) + NH2OH(3) <=> H2NO(88) + NH2OH(3)            0.2
120 : 2 H2NO(88) <=> HNO(87) + NH2OH(3)                     -0.1
207 : CH3OH(5) + OH(24) <=> CH2OH(45) + H2O(6)               0.1

Highest net rates of progress, surface
  0 : H2O(6) + X(1) <=> H2OX(30)                           1e+03
  3 : NH2OH(3) + X(1) <=> H3NOX(205)                       5e+02
  4 : HNO3(4) + X(1) <=> HNO3X(208)                        4e+02
  5 : CH3OH(5) + X(1) <=> CH4OX(211)                       3e+02
 71 : CH3OH(5) + 2 X(1) <=> CH3OX(210) + HX(26)            1e+02
  6 : NH2OH(3) + 2 X(1) <=> H2NOX(204) + HX(26)            1e+02
 42 : NH2OH(3) + 2 X(1) <=> H2NX

In [ ]:
r.thermo.TP  = 150, pressure
r.T



In [ ]:
sim.time

In [ ]:
gas.TDY = TDY
r.syncState()
r.thermo.T

In [ ]:
r.thermo.X - gas.X

In [ ]:
report_rate_constants()

In [ ]:
sim.verbose

In [ ]:
plt.barh(np.arange(len(gas.net_rates_of_progress)),gas.net_rates_of_progress)

In [ ]:
gas.T

In [ ]:
data = pd.read_csv(output_filename)
data

In [ ]:
def xlabels():
    ticks = []
    labels = []
    mm = 0
    while mm < length*1000:
        ticks.append( int(NReactors * mm * 0.001 / length ) )
        labels.append( str(mm) )
        mm += 1
    labels[-1] = labels[-1] + ' mm'
    plt.xticks(ticks, labels)
    plt.xlabel("Distance down reactor")

In [ ]:
data['T (C)'].plot()
plt.ylabel('T (C)')
xlabels()

In [ ]:
data[['NH2OH(3)', 'HNO3(4)', 'CH3OH(5)']].plot()
plt.ylabel('Mole fraction')
xlabels()

In [ ]:
list(data.columns)[:4]

In [ ]:
data[['T (C)', 'alpha']].plot()
xlabels()

In [ ]:
ax1 = data['T (C)'].plot()
plt.ylabel('Temperature (C)')
xlabels()
plt.legend()
ax2 = ax1.twinx()
data['alpha'].plot(ax=ax2, color='tab:orange')
ax2.set_ylim(-2, 2)
plt.legend()
plt.ylabel('alpha')
plt.tight_layout()
plt.savefig('temperature-and-alpha.pdf')
plt.show()

In [ ]:
data.columns

In [ ]:
data[['gas_heat','surface_heat']].plot()
#plt.ylim(-1e7, 1e7)
xlabels()
plt.savefig('gas_and_surface_heat.pdf')
plt.show()


In [ ]:
ax1 = data[['gas_heat','surface_heat']].plot()
plt.ylim(-1e9, 1e9)
xlabels()
plt.ylabel('Heat consumption rate (kJ/m3/s)')
plt.legend(loc='upper left')
ax2 = ax1.twinx()
data['alpha'].plot(ax=ax2, style='k:', alpha=0.5)
ax2.set_ylim(-10, 10)
plt.legend(loc='lower right')
plt.ylabel('alpha')
plt.tight_layout()
plt.savefig('heats-and-alpha.pdf')
plt.show()

In [ ]:
data[['T (C)']].plot()
plt.ylabel('Temperature (C)')
xlabels()
plt.tight_layout()
plt.savefig('temperature.pdf')
plt.show()

In [ ]:
data[['alpha']].plot(logy=True)
xlabels()

In [ ]:
data.plot(x='T (C)',y='alpha')


In [ ]:
specs = list(data.columns)
specs = specs[4:-3]
excluded = [s for s in data.columns if s not in specs]
gas_species = [s for s in specs if 'X' not in s ]
adsorbates = [s for s in specs if 'X' in s]

excluded, gas_species, adsorbates

In [ ]:
data[gas_species[0:5]].plot(logy=True, logx=True)

In [ ]:
for i in range(0,len(gas_species),10):
    data[gas_species[i:i+10]].plot(title='Gas mole fraction', logy=False)
    xlabels()
    plt.ylabel('Mole fraction')
    plt.tight_layout()
    plt.savefig(f'gas_mole_fractions_{i}.pdf')
    plt.show()
    


In [ ]:
for i in range(0,len(adsorbates),10):
    data[adsorbates[i:i+10]].plot(title='Surface coverages', logy=False)
    xlabels()
    plt.xlim(0,len(data)+5)
    plt.ylabel('Surface coverage')
    plt.tight_layout()
    plt.savefig(f'surface_coverages_{i}.pdf')
    plt.show()

In [ ]:
main_gas_species = data[gas_species].max().sort_values(ascending=False)[:10].keys()
data[main_gas_species].plot.area()

xlabels()
plt.xlim(0,len(data)+5)
plt.tight_layout()
plt.savefig(f'gas_mole_fractions_top10.pdf')
plt.show()

In [ ]:
main_adsorbates = data[adsorbates].max().sort_values(ascending=False)[:10].keys()
data[main_adsorbates].plot.area()

xlabels()
plt.xlim(190,len(data)+5)
plt.tight_layout()
plt.savefig(f'surface_coverages_top10.pdf')
plt.show()
    

In [ ]:
for a in main_adsorbates:
    s = surf.species(a)
    print(s, s.composition)

In [ ]:
surf.coverages

In [ ]:
surf.set_multiplier(1)
diagram = ct.ReactionPathDiagram(surf, 'X')
diagram.get_data()

In [ ]:
last_row = np.array(data.tail(1))
np.shape(last_row)

In [ ]:
output_filename